In [2]:
import tensorflow as tf
from tensorflow import keras
# 导入经典数据集模块
from tensorflow.keras import datasets
import matplotlib 
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [3]:
x = tf.linspace(-8, 8, 100)
y = tf.linspace(-8, 8, 100)
x, y = tf.meshgrid(x, y)
z = tf.sqrt(x**2 + y**2)
z = tf.sin(z) / z

In [4]:
fig = plt.figure()
ax = Axes3D(fig)
ax.contour3D(x.numpy(), y.numpy(), z.numpy(), 50)
plt.show()

<Figure size 640x480 with 0 Axes>

In [5]:
(x,y), (x_test, y_test)=datasets.mnist.load_data()

In [6]:
print("x：{0},y：{1},x_test：{2},y_test：{3}".format(x.shape, y.shape, x_test.shape, y_test.shape))

x：(60000, 28, 28),y：(60000,),x_test：(10000, 28, 28),y_test：(10000,)


In [7]:
# 转换为Dataset对象
train_db = tf.data.Dataset.from_tensor_slices((x, y))

In [8]:
# 随机打散，返回一个 Dataset 新对象
# Dataset.shuffle(buffer_size),其中buffer_size参数指定缓冲池的大小
train_db = train_db.shuffle(10000) #随机打散样本，但是不打散样本与标签的映射关系

In [9]:
# 批训练，同时计算多个样本
# 一次并行 128 个样本数据，数量大小根据显存配置
train_db = train_db.batch(128)

In [10]:
# 预处理，使得数据集的各式满足模型的输入要求
def preprocess(x, y):
    """
    自定义预处理函数
    参数：
    x -- 待处理的数据集,维度[b,28,28]
    y -- 待处理的数据集,[b]

    返回值：
    x -- 标准化和扁平化后的 x
    y -- 转换为 one_hot 向量
    """
    x = tf.cast(x, dtype=tf.float32) / 255. #标准化0-1
    x = tf.reshape(x, [-1, 28*28]) #扁平化

    y = tf.cast(x, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)

    return x,y

In [11]:
train_db = train_db.map(preprocess)

In [12]:
# 第一层
w1 = tf.Variable(tf.random.truncated_normal([784, 256], stddev=0.1))
b1 = tf.Variable(tf.zeros([256]))
# 第二层
w2 = tf.Variable(tf.random.truncated_normal([256, 128], stddev=0.1))
b2 = tf.Variable(tf.zeros([128]))
# 第三层
w3 = tf.Variable(tf.random.truncated_normal([128, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

In [ ]:
for epoch in range(20):
    for x,y in train_db:
        h1 = x @ w1 + b1
        h1 = tf.nn.relu(h1)
        h2 = h1 @ w2 +b2
        h2 = tf.nn.relu(h2)
        out = h2 @ w3 + b3
        pred = tf.argmax(out, axis=1)
        y = tf.argmax(y, axis=1)
        correct = tf.equal(pred, y)
        total_correct += tf.reduce_sum(tf.cast(correct, dtype=tf.int32)).numpy()
